# #161 Member Results
<i>Tabulate member results defined by beam shell resultants for multiple loadsets</i>
***

In [ ]:
# 
import numpy as np
import pandas as pd
import plotly.express as px
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("This script will extract results from an existing model, please open a model with results and run the script again.")

# Reference the current database for convenience
db = lusas.database()

In [ ]:
loadcase_ids = [1, 23, 27]  # just the max version of smart combs and envelopes
bss_name = "Girder 1"

In [ ]:
COMPONENTS = ["Fx", "Fy", "Fz", "Mx", "My", "Mz" ]

# Separate sheet for each component
df_results_list = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]

In [ ]:
beam_shell_slice : IFBeamShellSlice = win32.CastTo( db.getObject("Beam/Shell Slicing", bss_name), "IFBeamShellSlice" )

no_locations = beam_shell_slice.getNumberLocations()

results_positions = []
results = beam_shell_slice.getAllResults("max")
for i in range(len(results)):
    results_positions.append(results[i][6])

for iComp in range(6):
    df_results_list[iComp]["Position"] = results_positions

In [ ]:
def get_assoc_loadset(loadset:IFLoadset)->IFLoadset:
    if loadset.getTypeCode() == 3:
        return win32.CastTo(loadset, "IFEnvelope").getAssocLoadset()
    if loadset.getTypeCode() == 6:
        return win32.CastTo(loadset, "IFSmartCombination").getAssocLoadset()
    return loadset

In [ ]:
loadset_pairs = []
for id in loadcase_ids:   
    if not db.existsLoadset(id):
        print("loadcase missing")
        continue
    loadset = db.getLoadset(id)
    if loadset.needsPrimaryComponent():
        loadset_pairs.append((loadset, "max"))
        loadset_pairs.append((get_assoc_loadset(loadset), "min"))
    else:
        loadset_pairs.append((loadset, ""))

In [ ]:
totals_max = np.zeros((6, no_locations), dtype=float)
totals_min = np.zeros((6, no_locations), dtype=float)

In [ ]:

for loadset, max_min in loadset_pairs:
    
    if loadset.needsPrimaryComponent():

        for iComp, component, in enumerate(COMPONENTS):
            lusas.view().setActiveLoadsetAssocVal("Beam/Shell Slice Resultants", component, loadset)

            slice_results = np.array(beam_shell_slice.getAllResults("abs")).T
            # Copy results column to the dataframe
            df_results_list[iComp][f"{loadset.getName()} {max_min}"] = slice_results[iComp]
            if max_min == "max":
                totals_max[iComp] += slice_results[iComp]
            else:
                totals_min[iComp] += slice_results[iComp]

    else:
        lusas.view().setActiveLoadset(loadset)
        slice_results = np.array(beam_shell_slice.getAllResults("abs")).T
        # Copy to the dataframe
        for iComp in range(6):
            df_results_list[iComp][loadset.getName()] = slice_results[iComp]
            totals_max[iComp] += slice_results[iComp]
            totals_min[iComp] += slice_results[iComp]

In [ ]:

for iComp in range(6):
    df_results_list[iComp]["Totals max"] = totals_max[iComp]
    df_results_list[iComp]["Totals min"] = totals_min[iComp]

In [ ]:
with pd.ExcelWriter(f"Member Results - {bss_name}.xlsx") as writer:
    for iComp in range(6):
        df_results_list[iComp].to_excel(writer, sheet_name=COMPONENTS[iComp], index=False)

In [ ]:
for iComp in range(6):   
    fig = px.line(df_results_list[iComp], y=["Totals max", "Totals min"], x="Position", 
                  labels={'y':COMPONENTS[iComp], 'x':"Position"},
                    markers=True, title=f"Beam  {COMPONENTS[iComp]}")
    fig.show()